# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove colunas constantes
colsToRemove = []
for col in df.columns:
    if df[col].std() == 0:
        colsToRemove.append(col)
df.drop(colsToRemove, axis=1, inplace=True)

## Remove colunas multiplas
colsToRemove = []
columns = df.columns
for i in range(len(columns)-1):
    print(i, end="\r")
    v = df[columns[i]]
    for j in range(i+1,len(columns)):
        m = df.loc[(df[columns[j]]*v) != 0, [columns[j], columns[i]]]
        if m.shape[0] > 0:
            c = m.iloc[0,0]/m.iloc[0,1]
            if np.array_equal(c*v.values,df[columns[j]].values):
                colsToRemove.append(columns[j])
df.drop(colsToRemove, axis=1, inplace=True)

## Remove duplicate rows
df.drop_duplicates(inplace=True)

## Split (X,Y) 
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 287)

# Oversampling: Random Oversampling

## CNN 1 MAX RO

In [4]:
# info
model_name = 'CNN 1 MAX RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97446 samples, validate on 13684 samples
Epoch 1/10000
97446/97446 [==============================] - 33s - loss: 0.5399 - acc: 0.7317 - val_loss: 0.5835 - val_acc: 0.6869
Epoch 2/10000
97446/97446 [==============================] - 11s - loss: 0.4878 - acc: 0.7650 - val_loss: 0.4906 - val_acc: 0.7485
Epoch 3/10000
97446/97446 [==============================] - 12s - loss: 0.4650 - acc: 0.7782 - val_loss: 0.4493 - val_acc: 0.7827
Epoch 4/10000
97446/97446 [==============================] - 12s - loss: 0.4467 - acc: 0.7899 - val_loss: 0.5517 - val_acc: 0.7167
Epoch 5/10000
97446/97446 [==============================] - 12s - loss: 0.4314 - acc: 0.7994 - val_loss: 0.4040 - val_acc: 0.8093
Epoch 6/10000
97446/97446 [==============================] - 12s - loss: 0.4155 - acc: 0.8088 - val_loss: 0.5527 - val_acc: 0.7102
Epoch 7/10000
97446/97446 [==============================] - 12s - loss: 0.4042 - acc: 0.8158 - val_loss: 0.4266 - val_acc: 0.7982
Epoch 8/10000
97446/97446 [=

97522/97522 [==============================] - 11s - loss: 0.4204 - acc: 0.8049 - val_loss: 0.5633 - val_acc: 0.7208
Epoch 7/10000
97522/97522 [==============================] - 11s - loss: 0.4090 - acc: 0.8109 - val_loss: 0.4782 - val_acc: 0.7780
Epoch 8/10000
97522/97522 [==============================] - 11s - loss: 0.3972 - acc: 0.8163 - val_loss: 0.4835 - val_acc: 0.7748
Epoch 9/10000
97522/97522 [==============================] - 11s - loss: 0.3946 - acc: 0.8193 - val_loss: 0.4553 - val_acc: 0.7973
Epoch 10/10000
97522/97522 [==============================] - 11s - loss: 0.3879 - acc: 0.8273 - val_loss: 0.4419 - val_acc: 0.8052
Epoch 11/10000
97522/97522 [==============================] - 11s - loss: 0.3812 - acc: 0.8303 - val_loss: 0.3987 - val_acc: 0.8243
Epoch 12/10000
97522/97522 [==============================] - 11s - loss: 0.3748 - acc: 0.8338 - val_loss: 0.4748 - val_acc: 0.7901
Epoch 13/10000
97522/97522 [==============================] - 12s - loss: 0.3734 - acc: 0.8340

97522/97522 [==============================] - 11s - loss: 0.3127 - acc: 0.8658 - val_loss: 0.4934 - val_acc: 0.7938
Epoch 29/10000
97522/97522 [==============================] - 11s - loss: 0.3060 - acc: 0.8672 - val_loss: 0.4806 - val_acc: 0.8198
Epoch 30/10000
97522/97522 [==============================] - 11s - loss: 0.3004 - acc: 0.8678 - val_loss: 0.4572 - val_acc: 0.8134
Epoch 31/10000
97522/97522 [==============================] - 13s - loss: 0.3041 - acc: 0.8695 - val_loss: 0.4260 - val_acc: 0.8447
Epoch 32/10000
97522/97522 [==============================] - 12s - loss: 0.3082 - acc: 0.8674 - val_loss: 0.4837 - val_acc: 0.7979
Epoch 33/10000
97522/97522 [==============================] - 13s - loss: 0.3064 - acc: 0.8689 - val_loss: 0.4765 - val_acc: 0.8191
Epoch 34/10000
97522/97522 [==============================] - 12s - loss: 0.3008 - acc: 0.8710 - val_loss: 0.5121 - val_acc: 0.8126
Epoch 35/10000
97522/97522 [==============================] - 12s - loss: 0.2983 - acc: 0.8

In [5]:
print(model_name)

metrics_kf

CNN 1 MAX RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.957331,0.731311,0.731311,0.554849,0.701471,0.560296,0.836512,00:09:25.311300
KF_2,0.943242,0.749634,0.749634,0.551351,0.722233,0.545390,0.802973,00:06:40.868121
KF_3,0.950168,0.735125,0.735125,0.551858,0.705007,0.552092,0.821889,00:08:07.447176
KF_4,0.951586,0.752158,0.752158,0.561337,0.712676,0.572282,0.848856,00:08:07.038154
KF_5,0.946300,0.735823,0.735823,0.552681,0.697471,0.556100,0.831886,00:05:59.941602
KF_6,0.939207,0.747271,0.750594,0.545367,0.704155,0.533145,0.789661,00:05:19.654028
KF_7,0.951164,0.756434,0.759756,0.560874,0.724990,0.569071,0.838858,00:08:08.620472
KF_8,0.942631,0.705334,0.705334,0.539190,0.674597,0.524607,0.787951,00:07:09.290344
KF_9,0.942563,0.753841,0.753841,0.545053,0.714947,0.527425,0.774766,00:05:20.058078
KF_10,0.943677,0.729685,0.729685,0.548973,0.706565,0.543822,0.807788,00:05:53.306484


## CNN 2 MAX RO

In [6]:
# info
model_name = 'CNN 2 MAX RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97552 samples, validate on 13684 samples
Epoch 1/10000
97552/97552 [==============================] - 13s - loss: 0.6259 - acc: 0.6753 - val_loss: 0.5887 - val_acc: 0.6953
Epoch 2/10000
97552/97552 [==============================] - 12s - loss: 0.5584 - acc: 0.7260 - val_loss: 0.5628 - val_acc: 0.7349
Epoch 3/10000
97552/97552 [==============================] - 12s - loss: 0.5350 - acc: 0.7411 - val_loss: 0.5262 - val_acc: 0.7543
Epoch 4/10000
97552/97552 [==============================] - 12s - loss: 0.5203 - acc: 0.7471 - val_loss: 0.5416 - val_acc: 0.7329
Epoch 5/10000
97552/97552 [==============================] - 12s - loss: 0.5100 - acc: 0.7504 - val_loss: 0.4684 - val_acc: 0.7852
Epoch 6/10000
97552/97552 [==============================] - 12s - loss: 0.5009 - acc: 0.7552 - val_loss: 0.4887 - val_acc: 0.7628
Epoch 7/10000
97552/97552 [==============================] - 12s - loss: 0.4929 - acc: 0.7609 - val_loss: 0.5648 - val_acc: 0.7232
Epoch 8/10000
97552/97552 [=

97460/97460 [==============================] - 12s - loss: 0.4300 - acc: 0.7987 - val_loss: 0.4822 - val_acc: 0.7700
Epoch 24/10000
97460/97460 [==============================] - 12s - loss: 0.4272 - acc: 0.7988 - val_loss: 0.4760 - val_acc: 0.7868
Epoch 25/10000
97460/97460 [==============================] - 12s - loss: 0.4248 - acc: 0.8009 - val_loss: 0.5080 - val_acc: 0.7752
Epoch 26/10000
97460/97460 [==============================] - 12s - loss: 0.4240 - acc: 0.8020 - val_loss: 0.5159 - val_acc: 0.7681
Epoch 27/10000
97460/97460 [==============================] - 12s - loss: 0.4181 - acc: 0.8058 - val_loss: 0.4644 - val_acc: 0.7989
Epoch 28/10000
97460/97460 [==============================] - 12s - loss: 0.4275 - acc: 0.8004 - val_loss: 0.5036 - val_acc: 0.7730
Epoch 29/10000
97460/97460 [==============================] - 12s - loss: 0.4281 - acc: 0.8025 - val_loss: 0.4459 - val_acc: 0.7892
Epoch 30/10000
97460/97460 [==============================] - 12s - loss: 0.4213 - acc: 0.8

97560/97560 [==============================] - 12s - loss: 0.5107 - acc: 0.7456 - val_loss: 0.5013 - val_acc: 0.7463
Epoch 6/10000
97560/97560 [==============================] - 12s - loss: 0.5034 - acc: 0.7480 - val_loss: 0.5494 - val_acc: 0.7272
Epoch 7/10000
97560/97560 [==============================] - 12s - loss: 0.4973 - acc: 0.7515 - val_loss: 0.5378 - val_acc: 0.7371
Epoch 8/10000
97560/97560 [==============================] - 12s - loss: 0.4957 - acc: 0.7508 - val_loss: 0.5014 - val_acc: 0.7568
Epoch 9/10000
97560/97560 [==============================] - 11s - loss: 0.4867 - acc: 0.7559 - val_loss: 0.4508 - val_acc: 0.7744
Epoch 10/10000
97560/97560 [==============================] - 11s - loss: 0.4821 - acc: 0.7585 - val_loss: 0.4993 - val_acc: 0.7452
Epoch 11/10000
97560/97560 [==============================] - 11s - loss: 0.4779 - acc: 0.7608 - val_loss: 0.5380 - val_acc: 0.7217
Epoch 12/10000
97560/97560 [==============================] - 11s - loss: 0.4722 - acc: 0.7649 

97586/97586 [==============================] - 11s - loss: 0.3945 - acc: 0.8197 - val_loss: 0.5038 - val_acc: 0.7664
Epoch 49/10000
97586/97586 [==============================] - 11s - loss: 0.3959 - acc: 0.8187 - val_loss: 0.4223 - val_acc: 0.8049
Epoch 50/10000
97586/97586 [==============================] - 11s - loss: 0.3906 - acc: 0.8216 - val_loss: 0.4919 - val_acc: 0.7973
Epoch 51/10000
97586/97586 [==============================] - 11s - loss: 0.3929 - acc: 0.8218 - val_loss: 0.4279 - val_acc: 0.8286
Epoch 52/10000
97586/97586 [==============================] - 11s - loss: 0.3918 - acc: 0.8225 - val_loss: 0.5041 - val_acc: 0.7812
Epoch 53/10000
97586/97586 [==============================] - 11s - loss: 0.3950 - acc: 0.8207 - val_loss: 0.4609 - val_acc: 0.8121
Epoch 54/10000
97586/97586 [==============================] - 11s - loss: 0.3972 - acc: 0.8196 - val_loss: 0.4847 - val_acc: 0.8134
Epoch 55/10000
97586/97586 [==============================] - 12s - loss: 0.3953 - acc: 0.8

In [7]:
print(model_name)

metrics_kf

CNN 2 MAX RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.917878,0.754891,0.758213,0.547259,0.701734,0.540193,0.803367,00:08:05.394411
KF_2,0.902787,0.761228,0.761228,0.550591,0.718843,0.544271,0.802578,00:07:15.626296
KF_3,0.886360,0.790410,0.790410,0.546498,0.721899,0.529314,0.774796,00:04:41.910596
KF_4,0.921683,0.770182,0.770182,0.553804,0.725216,0.551491,0.811760,00:11:33.873506
KF_5,0.900059,0.750611,0.750611,0.538184,0.700583,0.504897,0.736911,00:06:23.582150
KF_6,0.884997,0.788757,0.788757,0.541612,0.721578,0.507996,0.734412,00:04:52.676181
KF_7,0.885639,0.782898,0.786220,0.543480,0.725738,0.514547,0.745462,00:05:16.588363
KF_8,0.895744,0.756205,0.756205,0.543598,0.708062,0.524915,0.772691,00:07:14.804046
KF_9,0.909866,0.758142,0.758142,0.540575,0.707916,0.512320,0.748980,00:09:00.657049
KF_10,0.923816,0.732172,0.735506,0.550584,0.710286,0.547248,0.811867,00:14:01.156710


## CNN 3 MAX RO

In [8]:
# info
model_name = 'CNN 3 MAX RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97500 samples, validate on 13684 samples
Epoch 1/10000
97500/97500 [==============================] - 12s - loss: 0.6468 - acc: 0.6509 - val_loss: 0.6613 - val_acc: 0.6593
Epoch 2/10000
97500/97500 [==============================] - 12s - loss: 0.6205 - acc: 0.6851 - val_loss: 0.5753 - val_acc: 0.6833
Epoch 3/10000
97500/97500 [==============================] - 12s - loss: 0.6125 - acc: 0.6913 - val_loss: 0.5137 - val_acc: 0.7072
Epoch 4/10000
97500/97500 [==============================] - 12s - loss: 0.6029 - acc: 0.6986 - val_loss: 0.6298 - val_acc: 0.5981
Epoch 5/10000
97500/97500 [==============================] - 12s - loss: 0.5836 - acc: 0.7035 - val_loss: 0.6136 - val_acc: 0.5620
Epoch 6/10000
97500/97500 [==============================] - 12s - loss: 0.5611 - acc: 0.7076 - val_loss: 0.5074 - val_acc: 0.7760
Epoch 7/10000
97500/97500 [==============================] - 12s - loss: 0.5456 - acc: 0.7251 - val_loss: 0.6472 - val_acc: 0.5573
Epoch 8/10000
97500/97500 [=

97588/97588 [==============================] - 12s - loss: 0.5129 - acc: 0.7450 - val_loss: 0.5500 - val_acc: 0.6603
Epoch 37/10000
97588/97588 [==============================] - 12s - loss: 0.5108 - acc: 0.7456 - val_loss: 0.5878 - val_acc: 0.6209
Epoch 38/10000
97588/97588 [==============================] - 12s - loss: 0.5090 - acc: 0.7469 - val_loss: 0.5139 - val_acc: 0.7097
Epoch 39/10000
97588/97588 [==============================] - 12s - loss: 0.5091 - acc: 0.7464 - val_loss: 0.5170 - val_acc: 0.7197
Epoch 40/10000
97588/97588 [==============================] - 12s - loss: 0.5085 - acc: 0.7482 - val_loss: 0.4991 - val_acc: 0.7221
Epoch 41/10000
97588/97588 [==============================] - 12s - loss: 0.5066 - acc: 0.7487 - val_loss: 0.4953 - val_acc: 0.7430
Epoch 42/10000
97588/97588 [==============================] - 12s - loss: 0.5087 - acc: 0.7487 - val_loss: 0.4673 - val_acc: 0.7539
Epoch 43/10000
97588/97588 [==============================] - 12s - loss: 0.5051 - acc: 0.7

97516/97516 [==============================] - 12s - loss: 0.6042 - acc: 0.6790 - val_loss: 0.5670 - val_acc: 0.6363
Epoch 6/10000
97516/97516 [==============================] - 12s - loss: 0.5851 - acc: 0.6854 - val_loss: 0.6054 - val_acc: 0.6131
Epoch 7/10000
97516/97516 [==============================] - 12s - loss: 0.5597 - acc: 0.7076 - val_loss: 0.5963 - val_acc: 0.5976
Epoch 8/10000
97516/97516 [==============================] - 12s - loss: 0.5521 - acc: 0.7126 - val_loss: 0.5409 - val_acc: 0.6207
Epoch 9/10000
97516/97516 [==============================] - 12s - loss: 0.5480 - acc: 0.7144 - val_loss: 0.5283 - val_acc: 0.6196
Epoch 10/10000
97516/97516 [==============================] - 12s - loss: 0.5430 - acc: 0.7201 - val_loss: 0.5142 - val_acc: 0.6350
Epoch 11/10000
97516/97516 [==============================] - 12s - loss: 0.5388 - acc: 0.7216 - val_loss: 0.5696 - val_acc: 0.5976
Epoch 12/10000
97516/97516 [==============================] - 12s - loss: 0.5382 - acc: 0.7228 

97508/97508 [==============================] - 12s - loss: 0.5687 - acc: 0.7115 - val_loss: 0.5652 - val_acc: 0.6502
Epoch 30/10000
97508/97508 [==============================] - 13s - loss: 0.5705 - acc: 0.7116 - val_loss: 0.5992 - val_acc: 0.6150
Epoch 31/10000
97508/97508 [==============================] - 13s - loss: 0.5683 - acc: 0.7140 - val_loss: 0.5679 - val_acc: 0.6330
Epoch 32/10000
97508/97508 [==============================] - 12s - loss: 0.5635 - acc: 0.7174 - val_loss: 0.5312 - val_acc: 0.6621
Epoch 33/10000
97508/97508 [==============================] - 12s - loss: 0.5689 - acc: 0.7121 - val_loss: 0.5252 - val_acc: 0.6581
Epoch 34/10000
97508/97508 [==============================] - 13s - loss: 0.5625 - acc: 0.7208 - val_loss: 0.6148 - val_acc: 0.6046
Epoch 35/10000
97508/97508 [==============================] - 13s - loss: 0.5535 - acc: 0.7248 - val_loss: 0.6697 - val_acc: 0.5484
Epoch 36/10000
97508/97508 [==============================] - 12s - loss: 0.5530 - acc: 0.7

97610/97610 [==============================] - 13s - loss: 0.6681 - acc: 0.6044 - val_loss: 0.5929 - val_acc: 0.6939
Epoch 2/10000
97610/97610 [==============================] - 12s - loss: 0.6307 - acc: 0.6585 - val_loss: 0.5981 - val_acc: 0.6546
Epoch 3/10000
97610/97610 [==============================] - 12s - loss: 0.5940 - acc: 0.6892 - val_loss: 0.5654 - val_acc: 0.7382
Epoch 4/10000
97610/97610 [==============================] - 12s - loss: 0.5624 - acc: 0.7107 - val_loss: 0.5317 - val_acc: 0.7518
Epoch 5/10000
97610/97610 [==============================] - 12s - loss: 0.5544 - acc: 0.7141 - val_loss: 0.5372 - val_acc: 0.7477
Epoch 6/10000
97610/97610 [==============================] - 12s - loss: 0.5477 - acc: 0.7135 - val_loss: 0.5380 - val_acc: 0.7388
Epoch 7/10000
97610/97610 [==============================] - 12s - loss: 0.5444 - acc: 0.7141 - val_loss: 0.5179 - val_acc: 0.7347
Epoch 8/10000
97610/97610 [==============================] - 12s - loss: 0.5398 - acc: 0.7165 - v

In [9]:
print(model_name)

metrics_kf

CNN 3 MAX RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.842665,0.795839,0.799161,0.537678,0.725634,0.477613,0.671840,00:11:22.707700
KF_2,0.826231,0.801251,0.801251,0.538243,0.727123,0.480772,0.677759,00:06:46.845634
KF_3,0.838836,0.787825,0.787825,0.533705,0.705628,0.466069,0.654828,00:11:08.156137
KF_4,0.827797,0.800798,0.800798,0.543843,0.729284,0.514058,0.743094,00:05:49.819990
KF_5,0.832099,0.784337,0.784337,0.534609,0.708195,0.471805,0.665877,00:06:56.060201
KF_6,0.841368,0.801517,0.801517,0.535710,0.720562,0.464566,0.646803,00:08:23.949087
KF_7,0.809453,0.780080,0.780080,0.528929,0.686679,0.432525,0.590897,00:07:55.408278
KF_8,0.807232,0.693626,0.693626,0.525352,0.663937,0.426052,0.583925,00:14:55.391228
KF_9,0.850165,0.793024,0.793024,0.535277,0.721578,0.458281,0.633996,00:17:22.988192
KF_10,0.844171,0.790882,0.794216,0.538053,0.726027,0.481189,0.679384,00:08:27.730995


# Oversampling: SMOTE

## CNN 1 MAX SM

In [10]:
# info
model_name = 'CNN 1 MAX SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97465 samples, validate on 13684 samples
Epoch 1/10000
97465/97465 [==============================] - 12s - loss: 0.4078 - acc: 0.8230 - val_loss: 0.2498 - val_acc: 0.8861
Epoch 2/10000
97465/97465 [==============================] - 11s - loss: 0.3289 - acc: 0.8640 - val_loss: 0.3801 - val_acc: 0.8144
Epoch 3/10000
97465/97465 [==============================] - 11s - loss: 0.3047 - acc: 0.8751 - val_loss: 0.3866 - val_acc: 0.8292
Epoch 4/10000
97465/97465 [==============================] - 11s - loss: 0.2912 - acc: 0.8815 - val_loss: 0.3232 - val_acc: 0.8544
Epoch 5/10000
97465/97465 [==============================] - 11s - loss: 0.2773 - acc: 0.8886 - val_loss: 0.3656 - val_acc: 0.8261
Epoch 6/10000
97465/97465 [==============================] - 12s - loss: 0.2698 - acc: 0.8913 - val_loss: 0.2956 - val_acc: 0.8688
Epoch 7/10000
97465/97465 [==============================] - 11s - loss: 0.2630 - acc: 0.8944 - val_loss: 0.3512 - val_acc: 0.8456
Epoch 8/10000
97465/97465 [=

97426/97426 [==============================] - 11s - loss: 0.2499 - acc: 0.9019 - val_loss: 0.1987 - val_acc: 0.9230
Epoch 11/10000
97426/97426 [==============================] - 11s - loss: 0.2483 - acc: 0.9025 - val_loss: 0.4031 - val_acc: 0.8213
Epoch 12/10000
97426/97426 [==============================] - 12s - loss: 0.2424 - acc: 0.9050 - val_loss: 0.2312 - val_acc: 0.8965
Epoch 13/10000
97426/97426 [==============================] - 11s - loss: 0.2404 - acc: 0.9063 - val_loss: 0.4038 - val_acc: 0.8128
Epoch 14/10000
97426/97426 [==============================] - 11s - loss: 0.2377 - acc: 0.9064 - val_loss: 0.2558 - val_acc: 0.8869
Epoch 15/10000
97426/97426 [==============================] - 11s - loss: 0.2349 - acc: 0.9082 - val_loss: 0.4617 - val_acc: 0.8305
Epoch 16/10000
97426/97426 [==============================] - 12s - loss: 0.2322 - acc: 0.9099 - val_loss: 0.2943 - val_acc: 0.8810
Epoch 17/10000
97426/97426 [==============================] - 11s - loss: 0.2305 - acc: 0.9

97608/97608 [==============================] - 11s - loss: 0.2671 - acc: 0.8936 - val_loss: 0.4369 - val_acc: 0.8135
Epoch 6/10000
97608/97608 [==============================] - 11s - loss: 0.2602 - acc: 0.8977 - val_loss: 0.3366 - val_acc: 0.8572
Epoch 7/10000
97608/97608 [==============================] - 11s - loss: 0.2568 - acc: 0.8993 - val_loss: 0.2692 - val_acc: 0.8792
Epoch 8/10000
97608/97608 [==============================] - 11s - loss: 0.2484 - acc: 0.9028 - val_loss: 0.3536 - val_acc: 0.8405
Epoch 9/10000
97608/97608 [==============================] - 11s - loss: 0.2460 - acc: 0.9041 - val_loss: 0.3105 - val_acc: 0.8663
Epoch 10/10000
97608/97608 [==============================] - 11s - loss: 0.2412 - acc: 0.9060 - val_loss: 0.3599 - val_acc: 0.8533
Epoch 11/10000
97608/97608 [==============================] - 11s - loss: 0.2386 - acc: 0.9062 - val_loss: 0.3103 - val_acc: 0.8769
Epoch 12/10000
97608/97608 [==============================] - 11s - loss: 0.2353 - acc: 0.9077 

97216/97531 [============================>.] - ETA: 0s


In [11]:
print(model_name)

metrics_kf

CNN 1 MAX SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.975112,0.759450,0.759450,0.562906,0.681872,0.578958,0.872287,00:07:06.116775
KF_2,0.972263,0.766061,0.766061,0.564447,0.705130,0.579397,0.861897,00:07:07.046345
KF_3,0.974860,0.750639,0.750639,0.565566,0.690238,0.582762,0.873060,00:07:38.266331
KF_4,0.972994,0.801879,0.801879,0.573539,0.689877,0.595077,0.887661,00:06:08.518420
KF_5,0.971793,0.768893,0.768893,0.551373,0.713862,0.548175,0.811365,00:07:09.473754
KF_6,0.973001,0.782250,0.782250,0.567670,0.724045,0.583149,0.858458,00:06:42.199869
KF_7,0.969871,0.781176,0.784499,0.570022,0.721305,0.587769,0.865430,00:05:16.419479
KF_8,0.976759,0.740235,0.740235,0.567759,0.680767,0.586745,0.882399,00:07:43.795860
KF_9,0.976345,0.770645,0.770645,0.567236,0.701081,0.584802,0.870938,00:09:20.509885
KF_10,0.977708,0.736483,0.739816,0.557566,0.707978,0.565195,0.841205,00:08:36.486065


## CNN 2 MAX SM

In [12]:
# info
model_name = 'CNN 2 MAX SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97594 samples, validate on 13684 samples
Epoch 1/10000
97594/97594 [==============================] - 12s - loss: 0.4967 - acc: 0.7757 - val_loss: 0.4563 - val_acc: 0.7963
Epoch 2/10000
97594/97594 [==============================] - 12s - loss: 0.3866 - acc: 0.8326 - val_loss: 0.3767 - val_acc: 0.8229
Epoch 3/10000
97594/97594 [==============================] - 12s - loss: 0.3640 - acc: 0.8447 - val_loss: 0.4027 - val_acc: 0.8042
Epoch 4/10000
97594/97594 [==============================] - 12s - loss: 0.3491 - acc: 0.8525 - val_loss: 0.3427 - val_acc: 0.8367
Epoch 5/10000
97594/97594 [==============================] - 12s - loss: 0.3411 - acc: 0.8574 - val_loss: 0.3539 - val_acc: 0.8376
Epoch 6/10000
97594/97594 [==============================] - 12s - loss: 0.3338 - acc: 0.8600 - val_loss: 0.3748 - val_acc: 0.8306
Epoch 7/10000
97594/97594 [==============================] - 12s - loss: 0.3296 - acc: 0.8630 - val_loss: 0.3040 - val_acc: 0.8546
Epoch 8/10000
97594/97594 [=

97635/97635 [==============================] - 12s - loss: 0.3290 - acc: 0.8583 - val_loss: 0.3259 - val_acc: 0.8520
Epoch 9/10000
97635/97635 [==============================] - 12s - loss: 0.3254 - acc: 0.8591 - val_loss: 0.3404 - val_acc: 0.8559
Epoch 10/10000
97635/97635 [==============================] - 12s - loss: 0.3227 - acc: 0.8612 - val_loss: 0.4291 - val_acc: 0.8158
Epoch 11/10000
97635/97635 [==============================] - 12s - loss: 0.3173 - acc: 0.8645 - val_loss: 0.3743 - val_acc: 0.8348
Epoch 12/10000
97635/97635 [==============================] - 12s - loss: 0.3151 - acc: 0.8657 - val_loss: 0.4766 - val_acc: 0.8202
Epoch 13/10000
97635/97635 [==============================] - 12s - loss: 0.3110 - acc: 0.8689 - val_loss: 0.3192 - val_acc: 0.8521
Epoch 14/10000
97635/97635 [==============================] - 12s - loss: 0.3076 - acc: 0.8683 - val_loss: 0.3131 - val_acc: 0.8623
Epoch 15/10000
97635/97635 [==============================] - 12s - loss: 0.3062 - acc: 0.87

97529/97529 [==============================] - 12s - loss: 0.2815 - acc: 0.8836 - val_loss: 0.3419 - val_acc: 0.8522
Epoch 29/10000
97120/97529 [============================>.] - ETA: 0sKF_6
Train on 97480 samples, validate on 13684 samples
Epoch 1/10000
97480/97480 [==============================] - 12s - loss: 0.5261 - acc: 0.7621 - val_loss: 0.3968 - val_acc: 0.7407
Epoch 2/10000
97480/97480 [==============================] - 12s - loss: 0.3989 - acc: 0.8277 - val_loss: 0.4420 - val_acc: 0.8188
Epoch 3/10000
97480/97480 [==============================] - 12s - loss: 0.3609 - acc: 0.8488 - val_loss: 0.3638 - val_acc: 0.8350
Epoch 4/10000
97480/97480 [==============================] - 12s - loss: 0.3484 - acc: 0.8547 - val_loss: 0.3707 - val_acc: 0.8343
Epoch 5/10000
97480/97480 [==============================] - 12s - loss: 0.3414 - acc: 0.8574 - val_loss: 0.4804 - val_acc: 0.7905
Epoch 6/10000
97480/97480 [==============================] - 12s - loss: 0.3339 - acc: 0.8605 - val_loss

97531/97531 [==============================] - 12s - loss: 0.2970 - acc: 0.8781 - val_loss: 0.4047 - val_acc: 0.8163
Epoch 16/10000
97531/97531 [==============================] - 14s - loss: 0.2951 - acc: 0.8801 - val_loss: 0.3209 - val_acc: 0.8536
Epoch 17/10000
97531/97531 [==============================] - 13s - loss: 0.2939 - acc: 0.8803 - val_loss: 0.3602 - val_acc: 0.8312
Epoch 18/10000
97531/97531 [==============================] - 12s - loss: 0.2927 - acc: 0.8820 - val_loss: 0.3570 - val_acc: 0.8388
Epoch 19/10000
97531/97531 [==============================] - 12s - loss: 0.2896 - acc: 0.8822 - val_loss: 0.3255 - val_acc: 0.8437
Epoch 20/10000
97531/97531 [==============================] - 12s - loss: 0.2893 - acc: 0.8823 - val_loss: 0.3672 - val_acc: 0.8398
Epoch 21/10000
97531/97531 [==============================] - 12s - loss: 0.2868 - acc: 0.8833 - val_loss: 0.3515 - val_acc: 0.8398
Epoch 22/10000
97531/97531 [==============================] - 12s - loss: 0.2856 - acc: 0.8

In [13]:
print(model_name)

metrics_kf

CNN 2 MAX SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.961239,0.778916,0.778916,0.555316,0.690748,0.563496,0.846508,00:12:58.397946
KF_2,0.956770,0.777324,0.777324,0.564501,0.708813,0.579057,0.859792,00:10:43.526609
KF_3,0.964113,0.758582,0.758582,0.566422,0.698356,0.583569,0.870297,00:16:57.606667
KF_4,0.952401,0.787085,0.790407,0.564836,0.712760,0.579191,0.858195,00:04:26.274300
KF_5,0.954044,0.757811,0.757811,0.555917,0.699422,0.563128,0.841752,00:05:54.316420
KF_6,0.955265,0.794425,0.794425,0.557836,0.729016,0.561038,0.825178,00:09:15.016150
KF_7,0.961639,0.780716,0.784038,0.566212,0.712879,0.581756,0.861484,00:12:10.628196
KF_8,0.955036,0.755037,0.755037,0.552629,0.687451,0.558257,0.840174,00:06:35.253983
KF_9,0.957706,0.782547,0.782547,0.561469,0.711311,0.572878,0.850678,00:09:11.762685
KF_10,0.961373,0.777950,0.777950,0.563074,0.707018,0.576668,0.857782,00:09:27.832983


## CNN 3 MAX SM

In [14]:
# info
model_name = 'CNN 3 MAX SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97510 samples, validate on 13684 samples
Epoch 1/10000
97510/97510 [==============================] - 13s - loss: 0.6106 - acc: 0.6678 - val_loss: 0.4717 - val_acc: 0.8611
Epoch 2/10000
97510/97510 [==============================] - 12s - loss: 0.4733 - acc: 0.7922 - val_loss: 0.4046 - val_acc: 0.8525
Epoch 3/10000
97510/97510 [==============================] - 12s - loss: 0.4331 - acc: 0.8182 - val_loss: 0.4624 - val_acc: 0.8356
Epoch 4/10000
97510/97510 [==============================] - 12s - loss: 0.4205 - acc: 0.8273 - val_loss: 0.3861 - val_acc: 0.8396
Epoch 5/10000
97510/97510 [==============================] - 12s - loss: 0.4115 - acc: 0.8310 - val_loss: 0.4010 - val_acc: 0.8380
Epoch 6/10000
97510/97510 [==============================] - 13s - loss: 0.4061 - acc: 0.8334 - val_loss: 0.3835 - val_acc: 0.8598
Epoch 7/10000
97510/97510 [==============================] - 12s - loss: 0.4026 - acc: 0.8366 - val_loss: 0.4315 - val_acc: 0.8211
Epoch 8/10000
97510/97510 [=

97610/97610 [==============================] - 12s - loss: 0.3579 - acc: 0.8558 - val_loss: 0.3184 - val_acc: 0.8658
Epoch 19/10000
97610/97610 [==============================] - 12s - loss: 0.3544 - acc: 0.8574 - val_loss: 0.3975 - val_acc: 0.8232
Epoch 20/10000
97610/97610 [==============================] - 12s - loss: 0.3555 - acc: 0.8572 - val_loss: 0.5257 - val_acc: 0.7752
Epoch 21/10000
97610/97610 [==============================] - 12s - loss: 0.3544 - acc: 0.8573 - val_loss: 0.4131 - val_acc: 0.8267
Epoch 22/10000
97610/97610 [==============================] - 12s - loss: 0.3530 - acc: 0.8589 - val_loss: 0.3418 - val_acc: 0.8612
Epoch 23/10000
97610/97610 [==============================] - 12s - loss: 0.3515 - acc: 0.8579 - val_loss: 0.3319 - val_acc: 0.8579
Epoch 24/10000
97610/97610 [==============================] - 12s - loss: 0.3524 - acc: 0.8575 - val_loss: 0.5372 - val_acc: 0.7910
Epoch 25/10000
97610/97610 [==============================] - 12s - loss: 0.3494 - acc: 0.8

97487/97487 [==============================] - 12s - loss: 0.3901 - acc: 0.8427 - val_loss: 0.3752 - val_acc: 0.8520
Epoch 22/10000
97487/97487 [==============================] - 12s - loss: 0.3872 - acc: 0.8415 - val_loss: 0.6146 - val_acc: 0.7426
Epoch 23/10000
97487/97487 [==============================] - 13s - loss: 0.3856 - acc: 0.8418 - val_loss: 0.3619 - val_acc: 0.8430
Epoch 24/10000
97487/97487 [==============================] - 12s - loss: 0.3815 - acc: 0.8431 - val_loss: 0.3354 - val_acc: 0.8595
Epoch 25/10000
97487/97487 [==============================] - 12s - loss: 0.3806 - acc: 0.8434 - val_loss: 0.4107 - val_acc: 0.8386
Epoch 26/10000
97487/97487 [==============================] - 12s - loss: 0.3799 - acc: 0.8449 - val_loss: 0.4164 - val_acc: 0.8380
Epoch 27/10000
97487/97487 [==============================] - 12s - loss: 0.3786 - acc: 0.8451 - val_loss: 0.3359 - val_acc: 0.8601
Epoch 28/10000
97487/97487 [==============================] - 12s - loss: 0.3746 - acc: 0.8

97466/97466 [==============================] - 13s - loss: 0.3950 - acc: 0.8449 - val_loss: 0.3592 - val_acc: 0.8260
Epoch 6/10000
97466/97466 [==============================] - 13s - loss: 0.3884 - acc: 0.8492 - val_loss: 0.4259 - val_acc: 0.8059
Epoch 7/10000
97466/97466 [==============================] - 13s - loss: 0.3844 - acc: 0.8502 - val_loss: 0.4804 - val_acc: 0.8112
Epoch 8/10000
97466/97466 [==============================] - 13s - loss: 0.3824 - acc: 0.8524 - val_loss: 0.4277 - val_acc: 0.8173
Epoch 9/10000
97466/97466 [==============================] - 13s - loss: 0.3783 - acc: 0.8531 - val_loss: 0.2636 - val_acc: 0.8707
Epoch 10/10000
97466/97466 [==============================] - 13s - loss: 0.3758 - acc: 0.8542 - val_loss: 0.3684 - val_acc: 0.8445
Epoch 11/10000
97466/97466 [==============================] - 13s - loss: 0.3674 - acc: 0.8562 - val_loss: 0.3707 - val_acc: 0.8081
Epoch 12/10000
97466/97466 [==============================] - 13s - loss: 0.3604 - acc: 0.8550 

In [15]:
print(model_name)

metrics_kf

CNN 3 MAX SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.939957,0.789402,0.789402,0.560293,0.699771,0.572189,0.854663,00:15:22.571701
KF_2,0.933482,0.804664,0.807986,0.561250,0.739319,0.566938,0.829672,00:06:50.004023
KF_3,0.938981,0.783674,0.783674,0.565866,0.719833,0.580243,0.856485,00:12:33.456070
KF_4,0.938324,0.790549,0.790549,0.558484,0.703617,0.567878,0.846751,00:13:57.850594
KF_5,0.935714,0.773936,0.773936,0.563986,0.697415,0.579404,0.865430,00:11:51.677876
KF_6,0.938584,0.799842,0.799842,0.563752,0.723619,0.575480,0.848461,00:05:21.285475
KF_7,0.941711,0.786018,0.786018,0.548692,0.714429,0.539988,0.797159,00:12:48.707616
KF_8,0.941322,0.762477,0.762477,0.547951,0.704975,0.541132,0.803473,00:06:43.682575
KF_9,0.938636,0.796769,0.800103,0.576382,0.687221,0.599062,0.893435,00:06:28.623357
KF_10,0.934456,0.797183,0.797183,0.561800,0.718777,0.572335,0.846599,00:10:44.964467


# Metrics

In [16]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
CNN 1 MAX RO Clean,0.946787,0.739662,0.740326,0.551153,0.706411,0.548423,0.814114,00:07:01.153575
CNN 2 MAX RO Clean,0.902883,0.764549,0.765547,0.545619,0.714186,0.527719,0.774282,00:07:50.626930
CNN 3 MAX RO Clean,0.832002,0.782918,0.783584,0.535140,0.711465,0.467293,0.654840,00:09:54.905744
CNN 1 MAX SM Clean,0.974071,0.765771,0.766437,0.564808,0.701616,0.579203,0.862470,00:07:16.883278
CNN 2 MAX SM Clean,0.957958,0.775039,0.775704,0.560821,0.705777,0.571904,0.851184,00:09:46.061593
CNN 3 MAX SM Clean,0.938117,0.788451,0.789117,0.560846,0.710897,0.569465,0.844213,00:10:16.282375
